## 1. 간단한 해쉬 예시

In [1]:
hash_table = [0 for i in range(10)]
print(hash_table)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [8]:
# 해쉬 함수로는 다양한 함수를 사용할 수 있지만, 가장 간단한 방법이 Division 기법임.
def hash_func(data):
    return data % 5

data1 = 'Andy'
data2 = 'Dave'
data3 = 'Trump'
data4 = 'Banana'

## ord(): 문자의 ASCII(아스키)코드 리턴
print(ord(data1[0])), print(ord(data2[0])), print(ord(data3[0])), print(ord(data4[0]))

65
68
84
66


(None, None, None, None)

In [9]:
def storage_data(data, value):
    key = ord(data[0])
    hash_address = hash_func(key)
    hash_table[hash_address] = value
    
storage_data('Andy', '01011112222')
storage_data('Dave', '01011113333')
storage_data('Trump', '01011114444')
storage_data('Banana', '01011115555')

In [10]:
hash_table

['01011112222', '01011115555', 0, '01011113333', '01011114444', 0, 0, 0, 0, 0]

In [11]:
def get_data(data):
    key = ord(data[0])
    hash_address = hash_func(key)
    return hash_table[hash_address]

In [12]:
get_data('Andy')

'01011112222'

## 2. 충돌 해결 알고리즘
해쉬 테이블의 가장 큰 문제는 충돌의 경우입니다. <br/> 
즉, key의 값은 다르지만 hash function을 통과해서 얻게 되는 hash address가 동일한 경우 hash table 상의 동일한 위치에 저장이 되게 되는 것입니다. 이를 충돌(collision)이라고 부릅니다.

## 2.1 Chaining 기법
- **개방 해슁 또는 Open Hashing 기법** 중 하나: 해쉬 테이블 저장공간 외의 공간을 활용하는 기법
- 충돌이 일어나면, 링크드 리스트라는 자료 구조를 사용해서, 링크드 리스트로 데이터를 추가로 뒤에 연결시켜서 저장하는 기법
- 원래는 링크드 리스트로 구현해야 하지만, 별도로 구현을 하기엔 또 복잡하므로 List를 이용해서 링크드 리스트와 비슷한 형식으로 구현하였습니다.

![image](https://user-images.githubusercontent.com/57930520/123796930-9d874180-d920-11eb-9df6-03d7385e7298.png)


In [16]:
hash_table = [0 for i in range(8)]

def get_key(data):
    return hash(data)

def hash_func(key):
    return key % 8

def save_data(data, value):
    # 만약 key가 1과 9라면 hash function을 통해 얻게 되는 값은 동일하므로
    # hash table 상에서 동일한 위치에 저장이 되게 된다. 즉 collision이 발생한다.
    # 따라서, open hashing을 사용하면 링크드 리스트로 데이터를 추가로 뒤에 연결해 저장하게 되는데
    # hash address 기준으로는 둘 다 1이므로 누가 어떤 것인지 알 수 없다.
    # 따라서, 저장할 때 1 / value, 9 / value 형태로 저장하고자 index_key를 별도로 보관한다.
    index_key = get_key(data)
    hash_address = hash_func(index_key)
    if hash_table[hash_address] != 0: # 데이터가 무언가 있다면
        for index in range(len(hash_table[hash_address])):
            if hash_table[hash_address][index][0] == index_key: # key가 겹치는 경우
                hash_table[hash_address][index][1] = value # 덮어쓰기 해버림
                return
        hash_table[hash_address].append([index_key, value])
    else:
        hash_table[hash_address] = [[index_key, value]]
        
def read_data(data):
    index_key = get_key(data)
    hash_address = hash_func(index_key)
    if hash_table[hash_address] != 0:
        for index in range(len(hash_table[hash_address])):
            if hash_table[hash_address][index][0] == index_key:
                return hash_table[hash_address][index][1]
        return None # 다 돌았는데 일치하는게 없었다.
    else: # 애초에 0이라는건 데이터가 없었다는 말이다.
        return None

In [17]:
print (hash('Dave') % 8)
print (hash('Da') % 8)
print (hash('Data') % 8)

5
2
2


In [18]:
save_data('Da', '00112233')
save_data('Data', '01346857')

In [19]:
hash_table # 동일한 2의 위치에 링크드 리스트처럼 연결된 것을 확인할 수 있음.

[0,
 0,
 [[-8566226137386631478, '00112233'], [6571745629798940666, '01346857']],
 0,
 0,
 0,
 0,
 0]

In [20]:
read_data('Da')

'00112233'

In [21]:
read_data('Data')

'01346857'

## 2.2. Linear Probing 기법
- **폐쇄 해슁 또는 Close Hashing 기법** 중 하나: 해쉬 테이블 저장공간 안에서 충돌 문제를 해결하는 기법
- 충돌이 일어나면, 해당 hash address의 다음 address부터 맨 처음 나오는 빈공간에 저장하는 기법
- 저장공간 활용도를 높이기 위한 기법

In [22]:
hash_table = [0 for i in range(8)]

def get_key(data):
    return hash(data)

def hash_func(key):
    return key % 8

def save_data(data, value):
    index_key = get_key(data)
    hash_address = hash_func(index_key)
    if hash_table[hash_address] != 0: # 데이터가 무언가 있다면
        for index in range(hash_address, len(hash_table)): # Hash table 전체를 돌아다니면서 진행
            if hash_table[index] == 0: # 해당 index에 비어있다면
                hash_table[index] = [index_key, value] # 그냥 저장
                return
            elif hash_table[index][0] == index_key: # 이미 해당 키랑 동일한 데이터가 있으면
                hash_table[index][1] = value # 갱신
                return
    else:
        hash_table[hash_address] = [index_key, value]
        

def read_data(data):
    index_key = get_key(data)
    hash_address = hash_func(index_key)
    if hash_table[hash_address] != 0: # 데이터가 무언가 있따면
        for index in range(hash_address, len(hash_table)):
            if hash_table[index] == 0:
                return None
            elif hash_table[index][0] == index_key:
                return hash_table[index][1]
    else:
        return None

In [23]:
save_data('Da', '00112233')
save_data('Data', '01346857')

In [24]:
hash_table # 둘다 hash address 기준으로 2지만, 한개가 뒤로 밀려나 3에 저장된 것을 확인할 수 있음.

[0,
 0,
 [-8566226137386631478, '00112233'],
 [6571745629798940666, '01346857'],
 0,
 0,
 0,
 0]

In [25]:
read_data('Da')

'00112233'

In [26]:
read_data('Data')

'01346857'

## 3. 해쉬 함수와 키 생성 함수
- 파이썬의 hash() 함수는 실행할 때마다, 값이 달라질 수 있음
- 유명한 해쉬 함수들이 있음: SHA(Secure Hash Algorithm, 안전한 해시 알고리즘)
- 어떤 데이터도 유일한 고정된 크기의 고정값을 리턴해주므로, 해쉬 함수로 유용하게 활용 가능

### 3.1 SHA1

In [27]:
import hashlib

data = 'test'.encode() # byte로 바꿔주는 것 => string은 유니코드나 특별한 코드 체계에 기반해서 연결이 되어 있는데
# 데이터를 byte 형태로 데이터를 풀어준 다음에 hash 함수에 넣어야만 hash 값이 추출이 될 수 있다.
# 이러한 작업을 하는 방법은 .encode()를 하거나 b'test' 와 같은 방법을 사용하면 byte로 바뀜.
hash_object = hashlib.sha1()
hash_object.update(data)
hex_dig = hash_object.hexdigest() # 16진수로 추출
print (hex_dig)

a94a8fe5ccb19ba61c4c0873d391e987982fbbd3


### 3.2 SHA256

In [28]:
import hashlib

data = 'test'.encode()
hash_object = hashlib.sha256()
hash_object.update(data)
hex_dig = hash_object.hexdigest()
print (hex_dig)
print(int(hex_dig, 16))

9f86d081884c7d659a2feaa0c55ad015a3bf4f1b2b0b822cd15d6c15b0f00a08
72155939486846849509759369733266486982821795810448245423168957390607644363272
